# pyAFQ: Automated Fiber Quantification in Python 

The core of the tractometry.org ecosystem is the pyAFQ software library. This
library uses inputs from various other programs to perform the delineation of
major white matter pathways and to quantify white matter tissue properties
along their lengths - tract profiles. The final output of the program includes
a tabular summary with these tract profiles for each subject in the dataset,
and one merged table that includes the tract profiles for all of the subjects. 

Along the way, the program produces many different intermediate data, including 
tractography results, maps of tissue properties, and visualizations, which 
can be used for quality assurance of the data and processing.

There are two ways to use pyAFQ: through a command line interface, and by 
writing Python code. Here, we will focus on the latter, using pyAFQ's 
Python Application Programming Interface (API).

## Setup 

We start with a few setup steps. We start by importing the functionality that 
we will use from pyAFQ

In [ ]:
import os
import os.path as op

pwd = op.dirname(os.getcwd())

os.environ["TEMPLATEFLOW_HOME"] = op.join(pwd, "..", "data_", "tractometry")
os.environ["DIPY_HOME"] = op.join(pwd, "..", "data_", "tractometry")
os.environ["AFQ_HOME"] = op.join(pwd, "..", "data_", "tractometry")


In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib
import plotly
import pandas as pd
from AFQ.api.group import GroupAFQ

## Set up tracking parameters 


We make create a `tracking_params` variable, which we will pass to the
GroupAFQ object which specifies that we want 25,000 seeds randomly
distributed in the white matter. We only do this to make this example 
faster and consume less space. We also set ``num_chunks`` to `True`,
which will use ray to parallelize the tracking across all cores.
This can be removed to process in serial, or set to use a particular
distribution of work by setting `n_chunks` to an integer number.



In [ ]:
tracking_params = dict(n_seeds=100000,
                       random_seeds=True,
                       rng_seed=2022,
                       trx=True,
                       num_chunks=True)

In [ ]:
myafq = GroupAFQ(
    bids_path=op.join('..', 'data_', 'tractometry', 'stanford_hardi'),
    preproc_pipeline='vistasoft',
    tracking_params=tracking_params,
    viz_backend_spec='plotly_no_gif')

## Recognizing the bundles and calculating tract profiles:
Typically, users of pyAFQ are interested in calculating not only an overall
map of the FA, but also the major white matter pathways (or bundles) and
tract profiles of tissue properties along their length. To trigger the
pyAFQ pipeline that calculates the profiles, users can call the
`export('profiles')` method:

```{alert}

Running the code below triggers the full pipeline of operations leading to the
computation of the tract profiles. Therefore, it takes a little while to run
(about 10 minutes, typically)

```


In [ ]:
myafq.export('profiles')

In [ ]:
bundle_html = myafq.export("all_bundles_figure")
plotly.io.show(bundle_html["01"][0])

We can even use altair to visualize the tract profiles in all
of the bundles. We provide a more customizable interface for visualizing
the tract profiles using altair.
Again, to make this plot, it is required that you install with
`pip install pyAFQ[plot]` so that you have the necessary dependencies.




In [ ]:
profiles_df = myafq.combine_profiles()

# Visualize tract profiles 

We will use the [dash](https://dash.plotly.com/) python library to construct a 
dashboard that displays the FA tract profiles of each one of the tracts:

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4('DTI FA Profiles'),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=list(profiles_df.tractID.unique()),
        value=["Left Arcuate", "Right Arcuate"],
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"),
    Input("checklist", "value"))
def update_line_chart(tracts):
    df = profiles_df
    mask = df.tractID.isin(tracts)
    fig = px.line(df[mask],
        x="nodeID", y="dti_fa", color='tractID')
    return fig


app.run_server(debug=True)